In [37]:
import os
import re
import pandas as pd
import psycopg2
import postgres_creds as cred
# Since changes were made in cred and our ipynb can't see new changes, we use Importlib to reload the module
import importlib
importlib.reload(cred)

<module 'postgres_creds' from '/Users/chewynguyen/Desktop/csv_postgres_connector/postgres_creds.py'>

In [38]:
# 1. add csv's in current directory to a list
# re.sub(r'[^\w\.]', '_', csv) substitutes all non word and num characters
def create_csv_list():
    csv_files = []
    for csv in os.listdir(os.getcwd()):
        if '.csv' in csv:
            old_csv_name = str(os.getcwd() + '/' + csv)
            csv = re.sub(r'[^\w\.]', '_', csv).lower()
            # 2. Check cwd, if csv in cwd, move csv to new folder
            new_csv_name = str(os.getcwd() + '/' + csv)
            if os.path.isfile(new_csv_name):
                pass
            else:
                # Rename the file
                os.rename(old_csv_name, new_csv_name)
            csv_files.append(csv)
    return csv_files

In [39]:
# Create new folder in current directory
def change_directory(csv_files,new_directory):
    try:
        os.mkdir(new_directory)
    except:
        pass
    # move files to new directory
    for csv in csv_files:
        mv = "mv '{0}' {1}".format(csv,new_directory)
        os.system(mv)
    return


In [40]:
#Create dictionary with csv name as key and df as value
def create_dict(csv_files):
    df_dict = {}
    for csv in csv_files:
        csv_path = str(os.getcwd() + '/' + new_directory + '/' + csv)
        df_dict[csv] = pd.read_csv(csv_path, index_col = 0)
    return df_dict

In [41]:
# cleans column names and changes pd datatypes to sql datatypes
def clean_columns(dataframe):
    dataframe_columns = [re.sub(r'[^\w\.]', '_', column_name).lower() for column_name in dataframe.columns]

# Replacing pd datatypes with sql datatypes
    replacements = {
        'timedelta64[ns]': 'varchar(255)',
        'object': 'varchar(255)',
        'float64': 'float',
        'bool': 'boolean',
        'int64': 'int',
        'datetime64': 'timestamp'}
    replaced_dtypes = dataframe.dtypes.replace(replacements)
    # table schema
    column_dtype = ", ".join("{} {}".format(col_name, dtype) for (col_name, dtype) in zip(dataframe_columns, replaced_dtypes))

    return dataframe_columns, column_dtype


In [42]:
# Connects to postgres
def connect_to_postgres(host,user,password,database):
    conn = psycopg2.connect(host = host,
    user = user, 
    password = password,
    database = database)

    return conn
# uploads csv to DB
def upload_csv_to_DB(conn,dataframe,key, dataframe_columns,column_dtype):
    cursor = conn.cursor()
    db_table_name = key.split('.')[0]
    #dataframe.columns as a str
    dataframe_columns_insertable = ', '.join(dataframe_columns)
    # create queries
    drop_table = 'DROP TABLE IF EXISTS ' + db_table_name
    create_table = 'CREATE TABLE ' + db_table_name + " (" + column_dtype + ")"
    insert_into_table = 'INSERT INTO ' + db_table_name + '(' + dataframe_columns_insertable + ')' \
    +' VALUES ( %s' % ', '.join(['%s'] * len(dataframe_columns)) +')'
        
    select_table = 'SELECT * FROM ' + db_table_name

    cursor.execute(drop_table)
    cursor.execute(create_table)
    # create insert into function
    for index, row in dataframe.iterrows():
        cursor.execute(insert_into_table,row)

    conn.commit()

    # Grant access to all users
    cursor.execute('GRANT SELECT, INSERT, UPDATE ON TABLE %s TO PUBLIC' % db_table_name)

    # Show table in DB
    cursor.execute(select_table)
    for each in cursor:
        print(each)
    

    cursor.close()

In [43]:
# Main.py
new_directory = "imported_csv"

# Create list of csv
csv_files = create_csv_list()

# Create new directory and move files
change_directory (csv_files, new_directory)

# Create dict
df_dict = create_dict(csv_files)

for key in df_dict:
# value
    dataframe = df_dict[key]
    dataframe_columns, column_dtype = clean_columns(dataframe)
    conn = connect_to_postgres(cred.host, cred.user, cred.password, cred.database)
    upload_csv_to_DB(conn, dataframe, key, dataframe_columns, column_dtype)


('New York', 'NY', 'New York', 36061, 'New York', 40.6943, -73.9249, 18713220, 10715, 'polygon', False, True, 'America/New_York', 1, 1840034016)
('Los Angeles', 'CA', 'California', 6037, 'Los Angeles', 34.1139, -118.4068, 12750807, 3276, 'polygon', False, True, 'America/Los_Angeles', 1, 1840020491)
('Chicago', 'IL', 'Illinois', 17031, 'Cook', 41.8373, -87.6862, 8604203, 4574, 'polygon', False, True, 'America/Chicago', 1, 1840000494)
('Miami', 'FL', 'Florida', 12086, 'Miami-Dade', 25.7839, -80.2102, 6445545, 5019, 'polygon', False, True, 'America/New_York', 1, 1840015149)
('Dallas', 'TX', 'Texas', 48113, 'Dallas', 32.7936, -96.7662, 5743938, 1526, 'polygon', False, True, 'America/Chicago', 1, 1840019440)
('Philadelphia', 'PA', 'Pennsylvania', 42101, 'Philadelphia', 40.0077, -75.1339, 5649300, 4554, 'polygon', False, True, 'America/New_York', 1, 1840000673)
('Houston', 'TX', 'Texas', 48201, 'Harris', 29.7863, -95.3889, 5464251, 1399, 'polygon', False, True, 'America/Chicago', 1, 18400209